In [ ]:
# To see orthographic projections of vacancies and interstitials for a set of cases (e.g., all Fe 20 keV cases)
# Summary statistics are also generated for each case and for the set althogehter
# Instructions: Set basepath and fileset to the location where the data files (.xyz and .in), then run (or play) this cell
# Input optons: file location and set, option to read *.in file for PKA hit location and direction, and rotation option
# Results appear below with statistics and orthographic projections of clustering followed by vacanices/interstitials
# Overall group results are printed at the end
# The results can be saved (and most of the example files already saved) by rightclicking and saving as a complete web page
# Two files are saved with the locations based on rotation option (.pickle) and no rotation (.norotpickle)
# The default rotation option is PKA, which mean that the PKA (aqua blue sphere on inters/vac plots) should be at lower Z, somewhat centered on X,Y
# The default colors are purple for the interstitials and orange for the vacanices
# Units are in cell lengths, all xyz scales are the same (e.g., 50x50x50) and determined by the energy
import glob, os, math, numpy as np

base_path = "/users/lepoire/downloads/" #path to data files
fileset=glob.glob(base_path+'005*.xyz') #set of files to be analyzed
get_input=True # those .in files. This should be done now that the size is based on the energy
rotate =1 # 0:no rotate; 1- PKA axis; 2- ellipsoid axis

casestats=[]
for filename in fileset:
    #Read input file? for xrec, yrec, zrec, theta, phi of PKA
    if get_input:
        fileinname = filename.replace("fpos","md") #"005-md-10-10.in"
        fileinname=fileinname.replace("xyz","in")
        filepickle=filename.replace("xyz","pickle")
        with open(fileinname, 'r') as content_file:
            content = content_file.read()
        istart=content.find("box(1)")
        length=float(content[istart+10:istart+19])
        istart=content.find("ncell(1)")
        iend=content.find("ncell(2)")-1       
        ndim=float(content[istart+10:iend])
        istart=content.find("xrec")
        xrec=float(content[istart+10:istart+15])
        istart=content.find("yrec")
        yrec=float(content[istart+10:istart+15])
        istart=content.find("zrec")
        zrec=float(content[istart+10:istart+15])
        istart=content.find("recen")
        Epka=float(content[istart+10:istart+18])/1000. # Energy in keV
        istart=content.find("rectheta")
        theta=float(content[istart+10:istart+17])
        istart=content.find("recphi")
        phi=float(content[istart+10:istart+17])
        dtheta=theta/57.3
        dphi=phi/57.3
        cphi=math.cos(dphi)
        sphi=math.sin(dphi)
        ctheta=math.cos(dtheta)
        stheta=math.sin(dtheta)
        
    celllength=length/ndim
    square=math.floor(20*(Epka/10)**0.5)
    fd=open(filename,'r')
    print ('File:', filename)
    f=math.floor(2.3)
 
    # Read data file
    header1 = fd.readline()
    header2 = fd.readline()
    header2=header2.strip()
    columns=header2.split()
    length=float(columns[7])
    
    data=[]
    side= (length-celllength)/2. 
    for line in fd:
        line = line.strip()
        columns = line.split()
        source = {}
        source['xpos'] = float(columns[1])
        source['ypos'] = float(columns[2])
        source['zpos'] = float(columns[3])
        source['xind'] = math.floor((source['xpos']+side)/celllength+0.5)  
        source['yind'] = math.floor((source['ypos']+side)/celllength+0.5)
        source['zind'] = math.floor((source['zpos']+side)/celllength+0.5)
        data.append(source)
    ixrec=math.floor((xrec+side)/celllength+0.5)
    iyrec=math.floor((yrec+side)/celllength+0.5)
    izrec=math.floor((zrec+side)/celllength+0.5)
    m=len(data)
    fd.close()

    #count number of atoms in crystal cells (expect 2 for fcc)
    n = math.floor(pow((m+1)/2,1/3))
    m=n*n*n
    a = [0] * m
    xind = [0] * m
    yind = [0] * m
    zind = [0] * m
 
    for s in data:
        indx=s['xind']*n*n+s['yind']*n+s['zind']
        xind[indx]= math.floor(indx/(n*n)) #s['xind']
        yind[indx]=math.floor ((indx-xind[indx]*n*n)/n) #s['yind']
        zind[indx]=indx-xind[indx]*n*n-yind[indx]*n #s['zind']
        a[indx]+=1
 
    # colect statistics
    aindx=-1
    void_fill=[]
    meanx, meany, meanz=0.0, 0.0, 0.0
    for val in a:
        aindx+=1
        if val==2:
            xx=1
        else:
            location={}
            location['indx']=aindx
            location['charge']=val-2
            location['x']=math.floor(aindx/(n*n)) #xind[aindx]
            location['y']=math.floor ((aindx-location['x']*n*n)/n) #yind[aindx]
            location['z']=aindx-location['x']*n*n-location['y']*n #zind[aindx]
            meanx+=location['x']
            meany+=location['y']
            meanz+=location['z']
            void_fill.append(location)
    meanx=meanx/len(void_fill)
    meany=meany/len(void_fill)
    meanz=meanz/len(void_fill)
    print ('Number of Vacancies and Interstitial cells:',len(void_fill))
    aindx=-1
    sumvx, sumvy, sumvz, sumfx,sumfy,sumfz, sumvd, sumfd, sumvc, sumfc, sumfd2, sumvd2 =(0.0,)*12
    for spot in void_fill:
        if spot['charge']>0:
            sumfx+=spot['x']*spot['charge']
            sumfy+=spot['y']*spot['charge']
            sumfz+=spot['z']*spot['charge']
            sumfc+=spot['charge']
        else:
            sumvx+=spot['x']*spot['charge']
            sumvy+=spot['y']*spot['charge']
            sumvz+=spot['z']*spot['charge']
            sumvc+=spot['charge']
    for spot in void_fill:
        if spot['charge']>0:
            fd2=(spot['x']-sumfx/sumfc)**2+(spot['y']-sumfy/sumfc)**2+(spot['z']-sumfz/sumfc)**2
            sumfd2+=spot['charge']*fd2
            sumfd+=spot['charge']*fd2**0.5
        else:
            vd2=(spot['x']-sumvx/sumvc)**2+(spot['y']-sumvy/sumvc)**2+(spot['z']-sumvz/sumvc)**2
            sumvd2+=abs(spot['charge'])*vd2
            sumvd+=abs(spot['charge'])*vd2**0.5
    stdfd=((sumfd2-sumfd**2/sumfc)/sumfc)**0.5
    stdvd=((sumvd2-sumvd**2/sumfc)/sumfc)**0.5
    sumfd=sumfd/sumfc
    sumvd=sumvd/sumfc
    sumfx=sumfx/sumfc
    sumvx=sumvx/sumfc
    sumfy=sumfy/sumfc
    sumvy=sumvy/sumfc
    sumfz=sumfz/sumfc
    sumvz=sumvz/sumfc
    
    dist_hit_center= ((sumvx+ixrec)**2+(sumvy+iyrec)**2+(sumvz+izrec)**2)**0.5
    
    print ('All distance units in crytal cell lengths')
    print ('Summary statistics for:     Interst.   Vacancies   PKA Hit')
    print ('Number of defects:              {0:.0f}        {1:.0f}'.format(sumfc, -sumvc))
    print ('Mean Radius     :              {0:.1f}       {1:.1f}       {2:.1f}'.format(sumfd, sumvd,dist_hit_center))
    print ('Radius Std Dev. :              {0:.1f}       {1:.1f}'.format(stdfd, stdvd))   
    print ('Mean X          :              {0:.1f}       {1:.1f}       {2:.1f}'.format(sumfx, -sumvx, ixrec))
    print ('Mean Y          :              {0:.1f}       {1:.1f}       {2:.1f}'.format(sumfy, -sumvy, iyrec))
    print ('Mean Z          :              {0:.1f}       {1:.1f}       {2:.1f}'.format(sumfz, -sumvz, izrec))
    print ('')
    print ('Comparison of directional vectors (xyz)')
    print ('Hit to vac.center   :          {0:.1f}       {1:.1f}       {2:.1f}'.format(-sumvx-ixrec, -sumvy-iyrec, -sumvz-izrec))  
    print ('Primary recoil      :          {0:.1f}       {1:.1f}       {2:.1f}'.format(stheta*cphi, stheta*sphi, ctheta))     

    #further stats
    distff=[0]*500
    distvv=[0]*500
    distfv=[0]*500
    numclusterf=[0]*500
    numclusterv=[0]*500
    distcluster=5
    for spot in void_fill:
        cluster=0
        for spot2 in void_fill:
            dist=math.floor(((spot['x']-spot2['x'])**2+(spot['y']-spot2['y'])**2+(spot['z']-spot2['z'])**2)**0.5)
            if dist>0:
                if spot['charge']>0 and spot2['charge']>0:
                    distff[dist]+=1
                    if dist<distcluster:
                        cluster+=1
                elif spot['charge']<0 and spot2['charge']<0:
                    distvv[dist]+=1
                    if dist<distcluster:
                        cluster+=1
                else:
                    distfv[dist]+=1
        if spot['charge']>0:
            numclusterf[math.floor(cluster)]+=1
        else:
            numclusterv[math.floor(cluster)]+=1
    
    #find ellipsoid axes
    import numpy as np
    import matplotlib.pyplot as plt

    # Fixing random state for reproducibility
    np.random.seed(19680803)
    zhat=[0]*3
    xhat=[0]*3
    yhat=[0]*3
    i=0
    imax=1000
    summax,xmax,ymax,zmax,xmin,ymin,zmin=(0.0,)*7
    summin=1e20
    while i<imax:
        x = 2.*np.random.rand(1)-1.
        y = 2.*np.random.rand(1)-1.
        z = 2.*np.random.rand(1)-1.
        testlen=math.sqrt(x*x+y*y+z*z)
        x=x/testlen
        y=y/testlen
        z=z/testlen
        sum=0.0
        for spot in void_fill:
            if spot['charge']==-1:
                sum+=((spot['x']-meanx)*x+(spot['y']-meany)*y+(spot['z']-meanz)*z)**2
        if sum>summax:
            summax=sum
            xmax=x
            ymax=y
            zmax=z
        if sum<summin:
            summin=sum
            xmin=x
            ymin=y
            zmin=z         
        i+=1
    zhat[0]=xmax
    zhat[1]=ymax
    zhat[2]=zmax
    yhat[0]=xmin
    yhat[1]=ymin
    yhat[2]=zmin
    xhat[0]=ymax*zmin-ymin*zmax
    xhat[1]=zmax*xmin-zmin*xmax
    xhat[2]=xmax*ymin-xmin*ymax

   
    print ('Major axis          :          {0:.1f}       {1:.1f}       {2:.1f}'.format(float(xmax),float(ymax),float(zmax)))
    
    print ('')
    print ('Vacancy ellipsoid standard deviatons:  Max:', (summax/sumfc)**0.5,'Min: ', (summin/sumfc)**0.5)
    #print ('Major axis: [x,y,z]:', float(xmax), ymax,zmax)
    #print (summin, xmin, ymin,zmin)
    #print (xhat) 
    filestats=[sumfc,sumfd,sumvd,stdfd,stdvd,dist_hit_center,float((summax/sumfc)**0.5), float((summin/sumfc)**0.5)]
    casestats.append(filestats)
    datavf=[]
    datavf.append(xhat)
    datavf.append(yhat)
    datavf.append(zhat)

    #rotate?
    if rotate==0:
        zhat[0]=0.0
        zhat[1]=0.0
        zhat[2]=1.0
        yhat[0]=0.0
        yhat[1]=1.0
        yhat[2]=0.0
        xhat[0]=1.0
        xhat[1]=0.0
        xhat[2]=0.0
    elif rotate==1:
        zhat[0]=stheta*cphi
        zhat[1]=stheta*sphi
        zhat[2]=ctheta
        xhat[0]=ctheta*cphi
        xhat[1]=ctheta*sphi
        xhat[2]=-stheta
        yhat[0]=-sphi
        yhat[1]=cphi
        yhat[2]=0
    # else use ellipsoid axes
    
    from sklearn.datasets import make_blobs
    void_fill_rot=[]
    clusterset=[]
    
    # now that stats are done add PKA location as extra point for graphics
    location={}
    location['indx']=-1
    location['charge']=10
    location['x']=ixrec
    location['y']=iyrec
    location['z']=izrec
    void_fill.append(location)
    minaxis=0
    maxaxis=0
    
    npts=len(void_fill) # added location of PKA
    xplot=[0]*npts
    yplot=[0]*npts
    zplot=[0]*npts
    cplot=[0]*npts
    colorplot=[0]*npts
    psi=0.0
    psiz=0.0
    ipt=0
    blobs, labels = make_blobs(n_samples=npts, n_features=3)
    blobs_3d, labels = make_blobs(n_samples=4, n_features=npts+2) # 2 for corners to make a true cube
    for loc in void_fill:
        location=[0]*3
        clusterpt=[0]*3
        x=loc['x']-meanx
        y=loc['y']-meany
        z=loc['z']-meanz
        if loc['charge']>8: # PKA location
            ar=10
        elif loc['charge']>=0: 
            ar=0
        else:
            ar=20
        tempx=x*xhat[0]+y*xhat[1]+z*xhat[2]
        tempy=x*yhat[0]+y*yhat[1]+z*yhat[2]
        tempz=x*zhat[0]+y*zhat[1]+z*zhat[2]
        tempxx=tempx*math.cos(psi)+tempy*math.sin(psi)
        location[1]=-tempx*math.sin(psi)+tempy*math.cos(psi)
        location[0]=tempxx*math.cos(psiz)+tempz*math.sin(psiz)
        location[2]=-tempxx*math.sin(psiz)+tempz*math.cos(psiz)
        blobs[ipt,0]=tempxx*math.cos(psiz)+tempz*math.sin(psiz)
        blobs[ipt,1]=-tempx*math.sin(psi)+tempy*math.cos(psi)
        blobs[ipt,2]=-tempxx*math.sin(psiz)+tempz*math.cos(psiz)
        blobs_3d[0,ipt]=tempxx*math.cos(psiz)+tempz*math.sin(psiz)
        blobs_3d[1,ipt]=-tempx*math.sin(psi)+tempy*math.cos(psi)
        blobs_3d[2,ipt]=-tempxx*math.sin(psiz)+tempz*math.cos(psiz)
        blobs_3d[3,ipt]=ar/20.
        if blobs_3d[0,ipt]>maxaxis:
            maxaxis=blobs_3d[0,ipt]
        elif blobs_3d[0,ipt]<minaxis:
            minaxis=blobs_3d[0,ipt]
        if blobs_3d[1,ipt]>maxaxis:
            maxaxis=blobs_3d[1,ipt]
        elif blobs_3d[1,ipt]<minaxis:
            minaxis=blobs_3d[1,ipt]
        if blobs_3d[2,ipt]>maxaxis:
            maxaxis=blobs_3d[2,ipt]
        elif blobs_3d[2,ipt]<minaxis:
            minaxis=blobs_3d[2,ipt]
        ipt+=1
        void_fill_rot.append(location)
        clusterpt=[tempx,tempy,tempz]
        clusterset.append(clusterpt )
        xplot.append(location[0])
        yplot.append(location[1])
        zplot.append(location[2])
        cplot.append(50.)
        colorplot.append(ar/20.)
   
    # set corners for proportional cube
    blobs_3d[0,npts+1]=maxaxis
    blobs_3d[0,npts]=minaxis
    blobs_3d[1,npts+1]=maxaxis
    blobs_3d[1,npts]=minaxis
    blobs_3d[2,npts+1]=maxaxis
    blobs_3d[2,npts]=minaxis
    blobs_3d[3,npts]=1.
    blobs_3d[3,npts+1]=1.

    import sys
    #!{sys.executable} -m pip install hdbscan

    def plot_clusters(data, algorithm, args, kwds,square):
        labels = algorithm(*args, **kwds).fit_predict(data)
        palette = sns.color_palette('deep', np.unique(labels).max() + 1)
        colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
        fig=plt.figure()
        ax1=plt.gca()
        ax1.figure.set_size_inches(7,7)
        plt.scatter(data.T[0], data.T[1], c=colors, **plot_kwds)
        plt.axis('square')
        plt.axis([-square, square, -square, square])
        plt.show()
        ax2=plt.gca()
        ax2.figure.set_size_inches(7,7)
        plt.scatter(data.T[0], data.T[2], c=colors, **plot_kwds)
        plt.axis('square')
        plt.axis([-square, square, -square, square])
        plt.show()
        
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import sklearn.cluster as cluster
    import time
    from sklearn.datasets import make_blobs
    import pandas as pd
    %matplotlib inline
    sns.set_context('poster')
    sns.set_color_codes()
    plot_kwds = {'alpha' : 0.25, 's' : 80, 'linewidths':0}
    
    #import hdbscan #problem importing: ImportError: numpy.core.multiarray failed to import
    df = pd.DataFrame(np.random.randn(10, 4), columns=['A', 'B', 'C', 'D'])
    plot_clusters(blobs, cluster.DBSCAN, (), {'eps':7.025},square)   #007:6.025
    #plot_clusters(df, hdbscan.HDBSCAN, (), {'min_cluster_size':15}) #This is the good one but couldn't get it to run


    #plot voids and fills
    import matplotlib.pyplot as plt
    ax=plt.gca()
    ax.figure.set_size_inches(7,7)
    plt.scatter(xplot, yplot, s=cplot, c=colorplot, alpha=0.5)
    plt.axis('square')
    plt.axis([-square, square, -square, square])
    plt.show()
    ax=plt.gca()
    ax.figure.set_size_inches(7,7)
    plt.axis([-square, square, -square, square])
    plt.scatter(xplot, zplot, s=cplot, c=colorplot, alpha=0.5)
    plt.show()

    import pickle
    with open(filepickle, 'wb') as f:  # save locations for later 3d interaction
        pickle.dump(blobs_3d, f, pickle.HIGHEST_PROTOCOL)
    filepickle2=filename.replace("xyz","picklenorot")
    with open(filepickle2, 'wb') as f:  # save locations with no rotation for later 3d interaction
        pickle.dump(void_fill, f, pickle.HIGHEST_PROTOCOL)
    
# Now do set statistics
x=[0]*10
x2=[0]*10
n=len(casestats)
for file in casestats:
    indx=0
    for p in file:
        x[indx]+=p
        x2[indx]+=p*p
        indx+=1
indx=0
for val in x:
    x2[indx]=((x2[indx]-x[indx]**2/n)/n)**0.5 
    x[indx]=val/n
    indx+=1


print ('Summary statistics over group')
print ('Number of files in group: ', n) 
print ('')
print ('Result                   Average     StdDev')
print ('Number of vacancies   :  {0:.1f}         {1:.1f}   '.format(x[0],x2[0]))
print ('Radius(Interstitials) :  {0:.1f}         {1:.1f}   '.format(x[1],x2[1]))
print ('Radius(Vacanies)      :  {0:.1f}         {1:.1f}   '.format(x[2],x2[2]))
print ('RadiusStd(Interstitia):  {0:.1f}         {1:.1f}   '.format(x[3],x2[3]))
print ('RadiusStd(Vacanies)   :  {0:.1f}         {1:.1f}   '.format(x[4],x2[4]))
print ('Distance hit to center:  {0:.1f}         {1:.1f}   '.format(x[5],x2[5]))
print ('Width(major axis)     :  {0:.1f}         {1:.1f}   '.format(x[6],x2[6]))
print ('width(minor axis)     :  {0:.1f}         {1:.1f}   '.format(x[7],x2[7]))



